# Análises em SQL - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


### Importando Database final

In [2]:
DROP TABLE IF EXISTS Origem;
DROP TABLE IF EXISTS Receita;
DROP TABLE IF EXISTS Ingrediente;
DROP TABLE IF EXISTS IngredienteComposto;
DROP TABLE IF EXISTS Sabor;
DROP TABLE IF EXISTS Classificacao;
DROP TABLE IF EXISTS Componente;
DROP TABLE IF EXISTS OrgaoPublico;
DROP TABLE IF EXISTS IngredientesDasReceitas;
DROP TABLE IF EXISTS ComponentesDosIngredientes;
DROP TABLE IF EXISTS Recomendacao;

CREATE TABLE Origem (
  Nome VARCHAR(30),
  Tipo VARCHAR(20),
  PRIMARY KEY (Nome)
) AS SELECT
  ORIGEM, TIPO
FROM CSVREAD('../data/processed/database/origem.csv');

CREATE TABLE Receita (
  Id INT,
  Nome VARCHAR(200),
  BancoOriginal VARCHAR(20),
  Origem VARCHAR(30),
  PRIMARY KEY (Id, BancoOriginal),
  FOREIGN KEY (Origem)
    REFERENCES Origem(Nome)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  IDRECEITA, NOMERECEITA, BANCOORIGINAL, ORIGEM
FROM CSVREAD('../data/processed/database/receita.csv');

CREATE TABLE Classificacao (
  Nome VARCHAR(50),
  CategoriaSuperior VARCHAR(50),
  PRIMARY KEY (Nome)
) AS SELECT
  NOME, CATEGORIASUPERIOR
FROM CSVREAD('../data/processed/database/classificacao.csv');

CREATE TABLE Ingrediente (
  Nome VARCHAR(80),
  Classificacao VARCHAR(50),
  PRIMARY KEY (Nome),
  FOREIGN KEY (Classificacao)
    REFERENCES Classificacao(Nome)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  NOME, CLASSIFICACAO
FROM CSVREAD('../data/processed/database/ingrediente.csv');

CREATE TABLE IngredienteComposto (
  IngredienteComposto VARCHAR(80),
  IngredienteOriginal VARCHAR(80),
  PRIMARY KEY (IngredienteComposto, IngredienteOriginal),
  FOREIGN KEY (IngredienteComposto)
    REFERENCES Ingrediente(Nome)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY (IngredienteOriginal)
    REFERENCES Ingrediente(Nome)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  INGREDIENTECOMPOSTO, INGREDIENTEORIGINAL
FROM CSVREAD('../data/processed/database/ingredientecomposto.csv');

CREATE TABLE Sabor (
  Id INT,
  Ingrediente1 VARCHAR(80),
  Ingrediente2 VARCHAR(80),
  Quantidade INT,
  PRIMARY KEY (Id),
  FOREIGN KEY (Ingrediente1)
    REFERENCES Ingrediente(Nome)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY (Ingrediente2)
    REFERENCES Ingrediente(Nome)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  ID, A, B, QUANTIDADE
  FROM CSVREAD('../data/processed/database/sabor.csv');

CREATE TABLE Componente (
  Id INT,
  Nome VARCHAR(300),
  Tipo VARCHAR(20),
  PRIMARY KEY (Id, Tipo)
) AS SELECT
  ID, NOME, TIPO
FROM CSVREAD('../data/processed/database/componente.csv');

CREATE TABLE OrgaoPublico (
  Nome VARCHAR(20),
  PRIMARY KEY (Nome)
) AS SELECT
  NOME
FROM CSVREAD('../data/processed/database/orgaopublico.csv');

CREATE TABLE IngredientesDasReceitas (
  IdReceita INT,
  Banco VARCHAR(20),
  Ingrediente VARCHAR(80),
  Quantidade DECIMAL(10, 3),
  Unidade VARCHAR(20),
  PRIMARY KEY (IdReceita, Banco, Ingrediente, Unidade),
  FOREIGN KEY (IdReceita, Banco)
    REFERENCES Receita(Id, BancoOriginal)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY (Ingrediente)
    REFERENCES Ingrediente(Nome)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  ID, BANCO, INGREDIENTE, QUANTIDADE, UNIDADE
FROM CSVREAD('../data/processed/database/ingredientesdasreceitas.csv') 
-- Muitos dados para importar. Utilizando menos para visualizar os comandos
-- As análises podem não ficar 100% precisas. Recomendamos importar todos os dados
-- para análises melhores, se a máquina aguentar milhões de entradas (apagar a prox. linha).
WHERE UNIDADE <> 'desconhecida' AND UNIDADE <> 'unidade' AND ID < 25000;

CREATE TABLE ComponentesDosIngredientes (
  Ingrediente VARCHAR(80),
  IdComponente INT,
  Tipo VARCHAR(15),
  Quantidade DECIMAL(10, 3),
  Unidade VARCHAR(30),
  PRIMARY KEY (Ingrediente, IdComponente, Tipo, Unidade),
  FOREIGN KEY (Ingrediente)
    REFERENCES Ingrediente(Nome)
      ON UPDATE NO ACTION
      ON DELETE NO ACTION,
  FOREIGN KEY (IdComponente, Tipo)
    REFERENCES Componente(Id, Tipo)
      ON UPDATE NO ACTION
      ON DELETE NO ACTION
) AS SELECT
  INGREDIENTE, COMPONENTE, TIPO, QUANTIDADE, UNIDADE
FROM CSVREAD('../data/processed/database/componentesdosingredientes.csv');

CREATE TABLE Recomendacao (
  Orgao VARCHAR(20),
  IdComponente INT,
  Tipo VARCHAR(15),
  FaixaEtaria VARCHAR(20),
  QuantidadeMg DECIMAL(20, 10),
  PRIMARY KEY (Orgao, IdComponente, FaixaEtaria, Tipo),
  FOREIGN KEY (Orgao)
    REFERENCES OrgaoPublico(Nome)
      ON UPDATE NO ACTION
      ON DELETE NO ACTION,
  FOREIGN KEY (IdComponente, Tipo)
    REFERENCES Componente(Id, Tipo)
      ON UPDATE NO ACTION
      ON DELETE NO ACTION
) AS SELECT
  ORGAO, ID, TIPO, FAIXAETARIA, QUANTIDADEMG
FROM CSVREAD('../data/processed/database/recomendacao.csv');


### Visualizando as Tabelas

In [3]:
SELECT * FROM Origem;


In [4]:
SELECT * FROM Receita LIMIT 10;


In [5]:
SELECT * FROM Ingrediente LIMIT 10;


In [6]:
SELECT * FROM IngredienteComposto LIMIT 10;


In [7]:
SELECT * FROM Sabor LIMIT 10;


In [8]:
SELECT * FROM Classificacao LIMIT 10;


In [9]:
SELECT * FROM Componente LIMIT 10;


In [10]:
SELECT * FROM OrgaoPublico LIMIT 10;


NIH_US

In [11]:
SELECT * FROM IngredientesDasReceitas LIMIT 10;


In [12]:
SELECT * FROM ComponentesDosIngredientes LIMIT 10;


In [13]:
SELECT * FROM Recomendacao LIMIT 10;


# Implementações das Perguntas de Pesquisa/Análise

### Relacionando ingredientes das receitas e componentes químicos

Em algumas questões de análise, relações entre receitas e componentes são feitas. Para fazer essas associações, é necessário relacionar os ingredientes de cada receita com os componentes a partir das unidades de medida. Analisando os valores possíveis:

In [14]:
SELECT DISTINCT Unidade
FROM ComponentesDosIngredientes;


In [15]:
SELECT DISTINCT Unidade
FROM IngredientesDasReceitas;


Assim, as unidades de medidas dos ingredientes, conforme criado nos notebooks anteriores base são `unidade`, para ingredientes com quantidades definidas pelo próprio ingrediente (por exemplo, 1 ovo), `grama`, para medidas em gramas, `mililitro`, para medidas em mililitros e `desconhecida` para medidas desconhecidas.

Relacionando essas unidades com as unidades dos componentes dos ingredientes do FooDB, foi possível identificar, resumidamente, as seguintes unidades:

* ug/g
* ug/kg
* ug/L
* mg/100g
* g/kg
* mg/kg
* mg/l
* ug/100g

Essas medidas podem ser associadas aos ingredientes nas receitas individualmente, respeitando a unidade (em gramas ou em mililitros) do componente e a unidade do componente. Expressões regulares podem ser usadas para identificar os padrões acima e, na construção dos dados dessa relação, todas as unidades dos componentes podem ser consideradas em miligramas por simplificação.

In [16]:
DROP VIEW IF EXISTS ComponentesUgG;
DROP VIEW IF EXISTS ComponentesUgKg;
DROP VIEW IF EXISTS ComponentesUgL;
DROP VIEW IF EXISTS ComponentesMg100G;
DROP VIEW IF EXISTS ComponentesGKg;
DROP VIEW IF EXISTS ComponentesMgKg;
DROP VIEW IF EXISTS ComponentesMgL;
DROP VIEW IF EXISTS ComponentesUg100G;

CREATE VIEW ComponentesUgG AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade / 1000000000) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'grama'
        AND (CI.Unidade LIKE 'ug/g%' OR CI.Unidade LIKE 'ug/ 100g%');

CREATE VIEW ComponentesUgKg AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade / 1000000000000) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'grama'
        AND CI.Unidade LIKE 'ug/kg%';

CREATE VIEW ComponentesUgL AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade / 1000000000000) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'mililitro'
        AND CI.Unidade LIKE 'ug/L';

CREATE VIEW ComponentesMg100G AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade / 100) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'grama'
        AND (CI.Unidade LIKE 'mg/100 g%' OR CI.Unidade LIKE 'mg/100g');

CREATE VIEW ComponentesGKg AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'grama'
        AND CI.Unidade LIKE 'g/kg%';

CREATE VIEW ComponentesMgKg AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade / 1000) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'grama'
        AND CI.Unidade LIKE 'mg/kg%';

CREATE VIEW ComponentesMgL AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade / 1000) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'mililitro'
        AND CI.Unidade LIKE 'mg/l';

CREATE VIEW ComponentesUg100G AS
    SELECT IR.IdReceita, IR.Banco, IR.Ingrediente, CI.IdComponente, CI.Tipo, (CI.Quantidade * IR.Quantidade / 100000000000) QuantidadeMg
    FROM IngredientesDasReceitas IR, ComponentesDosIngredientes CI
    WHERE IR.Ingrediente = CI.Ingrediente
        AND IR.Unidade = 'grama'
        AND CI.Unidade LIKE 'ug/ 100g%';


Visualizando:

In [17]:
SELECT * FROM ComponentesUgG LIMIT 10;


In [18]:
SELECT * FROM ComponentesUgKg LIMIT 10;


In [19]:
SELECT * FROM ComponentesUgL LIMIT 10;


In [20]:
SELECT * FROM ComponentesMg100G LIMIT 10;


In [21]:
SELECT * FROM ComponentesGKg LIMIT 10;


In [22]:
SELECT * FROM ComponentesMgKg LIMIT 10;


In [23]:
SELECT * FROM ComponentesMgL LIMIT 10;


In [24]:
SELECT * FROM ComponentesUg100G LIMIT 10;


Agora, é possível unir todas essas VIEWs em uma única, que contém a quantidade em miligramas de cada componente para cada ingrediente de cada receita:

In [25]:
DROP VIEW IF EXISTS ComponentesDosIngredientesDasReceitas;

CREATE VIEW ComponentesDosIngredientesDasReceitas AS
    SELECT * FROM ComponentesUgG
    UNION
    SELECT * FROM ComponentesUgKg
    UNION
    SELECT * FROM ComponentesUgL
    UNION
    SELECT * FROM ComponentesMg100G
    UNION
    SELECT * FROM ComponentesGKg
    UNION
    SELECT * FROM ComponentesMgKg
    UNION
    SELECT * FROM ComponentesMgL
    UNION
    SELECT * FROM ComponentesUg100G;


Visualizando:

In [26]:
SELECT * FROM ComponentesDosIngredientesDasReceitas LIMIT 10;


Exportando esses dados:

In [27]:
-- Operação custosa
-- CALL CSVWRITE('../data/processed/componentesingredientesreceitas.csv',
-- 'SELECT * FROM ComponentesDosIngredientesDasReceitas');


### 1) Quais os componentes químicos mais presentes nas receitas em cada região do mundo?

##### Analisando as receitas das regiões do mundo, buscamos descobrir quais componentes químicos são prevalecentes na culinária da região.

Para essa análise, primeiramente criaremos uma VIEW com tuplas (País/RegiãoDoMundo, Componente, QtdEmGramas), considerando todas as receitas que existem naquela região. A partir da VIEW criada ComponentesDosIngredientesDasReceitas, é possível agrupar pela chave primária dos componentes e pela região, para termos exatamente as tuplas que queremos.

In [28]:
DROP VIEW IF EXISTS QtdComponentesPorRegiao;

CREATE VIEW QtdComponentesPorRegiao AS
    SELECT RE.Origem, C.Nome, SUM(CO.quantidadeMg) Qtd
    FROM Receita RE
    JOIN ComponentesDosIngredientesDasReceitas CO
        ON RE.Id = CO.IdReceita AND RE.BancoOriginal = CO.Banco
    JOIN Componente C
        ON C.Id = CO.IdComponente AND C.Tipo = CO.Tipo
    JOIN Origem O
        ON O.Nome = RE.Origem
    WHERE O.Tipo = 'Regiao' AND C.Tipo = 'Compound'
    GROUP BY RE.Origem, C.Nome;


Visualizando:

In [29]:
SELECT * FROM QtdComponentesPorRegiao ORDER BY Origem LIMIT 10;


Agora, para cada região do mundo, podemos visualizar, por exemplo, os 5 componentes químicos mais presentes em receitas daquela região.

In [71]:
SELECT Nome, Qtd
FROM QtdComponentesPorRegiao
WHERE Origem = 'Korea'
ORDER BY Qtd DESC
LIMIT 10;


É possível também criar uma VIEW com o componente mais usado para cada região. Quando a região de uma receita é NULL, isso significa que a origem dela é desconhecida.

In [31]:
DROP VIEW IF EXISTS MaiorUsoPorRegiao;

CREATE VIEW MaiorUsoPorRegiao AS
  SELECT Q1.Origem, Q1.Nome
  FROM QtdComponentesPorRegiao Q1,
       (SELECT Origem, MAX(Qtd) Qtd
        FROM QtdComponentesPorRegiao
        GROUP BY Origem) Q2
  WHERE Q1.Origem = Q2.Origem AND Q1.Qtd = Q2.Qtd;


Visualizando:

In [32]:
-- SELECT * FROM MaiorUsoPorRegiao LIMIT 10;


### 2) Quais são as regiões do mundo que utilizam mais de um certo ingrediente?

##### Dado um ingrediente específico, buscamos analisar que região utiliza esse ingrediente com mais frequência

Analisaremos a frequência do dado ingrediente para todas as receitas de determinada região do mundo. Realizaremos um SELECT para obtermos uma tabela com tuplas na forma (RegiaoDoMundo, Freq), em que Freq é a frequência com que o ingrediente aparece nas receitas dessa região, e ordenaremos as tuplas.

* Realizamos um JOIN para alinharmos corretamente as tuplas das tabelas e para selecionarmos o ingrediente específico
* Agrupamos as tuplas por Região do Mundo e ordenamos os resultados

In [33]:
DROP VIEW IF EXISTS MaiorUsoAlhoRegioes;

CREATE VIEW MaiorUsoAlhoRegioes AS
    SELECT RE.Origem, IR.Ingrediente, COUNT(*) Freq
    FROM Receita RE
    JOIN IngredientesDasReceitas IR
       ON RE.Id = IR.IdReceita AND Re.BancoOriginal = IR.Banco
    JOIN Origem O
       ON O.Nome = RE.Origem
    WHERE IR.Ingrediente = 'Garlic' AND O.Tipo = 'Regiao'
    GROUP BY RE.Origem
    ORDER BY Freq DESC;


Visualizando:

In [34]:
SELECT * FROM MaiorUsoAlhoRegioes LIMIT 10;


### 3) A partir de uma necessidade alimentar de componentes químicos, quais são os pratos que têm ingredientes com as maiores quantidades de todos eles?

##### Dado componentes químicos específicos, buscamos descobrir quais receitas suprem melhor uma necessidade alimentar desse componente.

Analisaremos a quantidade de cada componente para cada receita. Depois, verificaremos quais receitas suprem as quantidades dos seguintes componentes:

* Menos de 2000 mg de Sodium
* De 0.01 mg a 0.03 mg de Vitamin D

In [35]:
DROP VIEW IF EXISTS ComponentesDasReceitas;

CREATE VIEW ComponentesDasReceitas AS
    SELECT CIR.IDReceita, CIR.Banco, CIR.IdComponente, CIR.Tipo, SUM(CIR.QuantidadeMg) Quantidade
    FROM ComponentesDosIngredientesDasReceitas CIR
    GROUP BY CIR.IDReceita, CIR.Banco, CIR.IdComponente, CIR.Tipo;


Visualizando:

In [36]:
SELECT * FROM ComponentesDasReceitas LIMIT 10;


Criando VIEWS para representar as receitas com quantidades desejadas dos componentes:

In [37]:
DROP VIEW IF EXISTS ReceitaSodio;
DROP VIEW IF EXISTS ReceitaVitaminaD;

CREATE VIEW ReceitaSodio AS
    SELECT CR.IdReceita, CR.Banco
    FROM ComponentesDasReceitas CR, Componente C
    WHERE CR.IdComponente = C.Id AND CR.Tipo = C.Tipo
        AND C.Nome = 'Sodium' AND CR.Quantidade < 2000;

CREATE VIEW ReceitaVitaminaD AS
    SELECT CR.IdReceita, CR.Banco
    FROM ComponentesDasReceitas CR, Componente C
    WHERE CR.IdComponente = C.Id AND CR.Tipo = C.Tipo
        AND C.Nome = 'Vitamin D' AND CR.Quantidade > 0.01 AND CR.Quantidade < 0.03;


Visualizando:

In [38]:
SELECT * FROM ReceitaSodio LIMIT 10;


In [39]:
SELECT * FROM ReceitaVitaminaD LIMIT 10;


Encontrando as receitas que respeitam as categorias:

In [40]:
SELECT *
FROM (
    SELECT * FROM ReceitaSodio
    INTERSECT
    SELECT * FROM ReceitaVitaminaD
)
LIMIT 10;


### 4) Quanto é necessário consumir de um alimento X para conseguir uma quantidade Y de um nutriente?

##### Analisando os dados buscamos saber a quantidade de uma receita que será necessário para atingir uma meta nutricional.

Os dados dos componentes de cada receita estão disponíveis na VIEW ComponentesDasReceitas. Assim, basta definir um nutriente Y e sua quantidade e uma receita X. Considerando o nutriente "Proteins" e uma quantidade de 10000 mg, bem como a receita de ID 6220 original do RecipeNLG:

In [41]:
SELECT (10000 / CIR.Quantidade) Quantidade
FROM ComponentesDasReceitas CIR, Componente C
WHERE CIR.IdComponente = C.Id AND CIR.Tipo = C.Tipo
      AND C.Nome = 'Proteins'
      AND CIR.IDReceita = 6220 AND CIR.Banco = 'RecipeNLG';


7.813079374506159356087336

### 5) Dado um grupo de classificação de ingredientes, qual país tem mais receitas com ingredientes desse grupo?

##### Com a análise de dados buscamos descobrir que pais utiliza mais de um grupo de alimentos em suas receitas.

Realizaremos um SELECT para criarmos uma tabela com tuplas no formato (PaísDoMundo, GrupoIngrediente, Freq), em que Freq é a frequência com que um determinado grupo de classificação de ingredientes aparece em todas as receitas daquele país.
* Realizamos um JOIN para alinharmos corretamente as tuplas das tabelas
* Passo a passo

In [55]:
DROP VIEW IF EXISTS FreqGrupoIngredientesPorRegiao;

CREATE VIEW FreqGrupoIngredientesPorRegiao AS
    SELECT RE.Origem, IG.Classificacao, COUNT(DISTINCT(RE.Id, RE.BancoOriginal)) Freq
    FROM Receita RE
    JOIN IngredientesDasReceitas IR
        ON RE.Id = IR.IdReceita AND RE.BancoOriginal = IR.Banco
    JOIN Ingrediente IG
        ON IR.Ingrediente = IG.Nome
    JOIN Origem ORI
        ON RE.Origem = ORI.Nome AND ORI.Tipo = 'Pais'
    GROUP BY RE.Origem, IG.Classificacao;


Visualizando:

In [56]:
SELECT * FROM FreqGrupoIngredientesPorRegiao LIMIT 10;


Considerando o grupo "Herbs", por exemplo, os 5 países que mais utilizaram esse grupo (em total de receitas) são:

In [57]:
SELECT *
FROM FreqGrupoIngredientesPorRegiao
WHERE Classificacao = 'Herbs'
ORDER BY Freq DESC
LIMIT 5;


Calcularemos também, para cada país, a % de receitas que utilizam esse ingrediente. Como podemos ter muitas receitas de um país (ex. USA) e poucas receitas de outro país (ex. Japan), é interessante também analisarmos a % de receitas em relação ao total de cada país para podermos ter análises um pouco mais justas. Primeiramente, criaremos uma VIEW com tuplas (País, Qtd), em que Qtd é a quantidade total de receitas por país do mundo.

In [58]:
DROP VIEW IF EXISTS QtdReceitasPais;

CREATE VIEW QtdReceitasPais AS
    (SELECT RE.Origem, COUNT(*) Tot
     FROM Receita RE
     JOIN Origem O
        ON RE.Origem = O.Nome AND O.Tipo = 'Pais'
     GROUP BY RE.Origem);


Visualizando:

In [59]:
SELECT * FROM QtdReceitasPais LIMIT 10;


Agora, com a VIEW FreqGrupoIngredientesPorRegiao, podemos realizar a divisão e calcular a porcentagem em cada caso, criando uma tabela com tuplas no formato (País, Classificacao, Porcentagem), em que Porcentagem será calculada pela frequência da classificação dividida pelo total de receitas multiplicado por 100.

In [60]:
DROP VIEW IF EXISTS PorcentagemClassificacaoPorPais;

CREATE VIEW PorcentagemClassificacaoPorPais AS
    SELECT FR.Origem, FR.Classificacao, (100.0 * FR.Freq / QT.Tot) Porcentagem
    FROM FreqGrupoIngredientesPorRegiao FR
    JOIN QtdReceitasPais QT
        ON FR.Origem = QT.Origem;


Visualizando:

In [61]:
SELECT * FROM PorcentagemClassificacaoPorPais;


Agora, finalmente, podemos refazer a análise anterior que usou apenas as frequências e analisarmos os países com maiores porcentagens de "Herbs".

In [62]:
SELECT *
FROM PorcentagemClassificacaoPorPais
WHERE Classificacao = 'Herbs'
ORDER BY Porcentagem DESC
LIMIT 5;


### 6) Qual região do mundo é mais propensa a ter problemas de altos consumos de sódio?

##### Analisando os dados, queremos descobrir que país consome uma maior quantidade de sódio diariamente e, portanto, é mais propenso a ter problemas de alto consumo de sódio, como, por exemplo, hipertensão

Primeiramente, levamos em conta que, por dia, uma pessoa consome 3 refeições. Para cada região do mundo, contaremos quantas receitas com ingredientes em gramas ele tem no total. Criaremos uma VIEW no formato (Região do Mundo, Tot), em que Tot é o total de receitas daquela região com pelo menos um ingrediente em gramas. Após isso, calcularemos o consumo total de sódio (TotNa) na região do mundo, para cada ingrediente usando a informação da tabela ComponentesDosIngredientes Quantidade(mg/100g). Feito isso, poderemos calcular o sódio médio por refeição naquela região do mundo, fazendo TotNa/Tot. Considerando que uma pessoa consome 3 refeições 
* Criar a VIEW no formato (Região do Mundo, Tot), em que Tot é o total de receitas daquela região com pelo menos um ingrediente em gramas

In [65]:
DROP VIEW IF EXISTS MediaSodioRegiao;

CREATE VIEW MediaSodioRegiao AS
    SELECT R.Origem, COUNT(DISTINCT (CDR.IdReceita, CDR.Banco)) TotalReceitas, SUM(CDR.Quantidade) Quantidade
    FROM ComponentesDasReceitas CDR, Receita R, Componente C
    WHERE CDR.IdReceita = R.Id AND CDR.Banco = R.BancoOriginal
          AND CDR.IdComponente = C.ID AND C.Nome = 'Sodium'
    GROUP BY R.Origem;


Visualizando:

In [66]:
SELECT * FROM MediaSodioRegiao LIMIT 10;


* Calcular a quantidade média de sódio por refeição por regiões (TotNa/Tot).

In [67]:
DROP VIEW IF EXISTS NaMedioPorRefeicaoEmMgPorRegiao;

CREATE VIEW NaMedioPorRefeicaoEmMgPorRegiao AS
    SELECT Origem, (Quantidade / TotalReceitas) MediaSodio
    FROM MediaSodioRegiao;


Visualizando:

In [68]:
SELECT * FROM NaMedioPorRefeicaoEmMgPorRegiao LIMIT 10;


* Agora, considerando uma pessoa entre 19-30 anos, vamos selecionar as regiões do mundo cujo consumo médio diário de sódio seja maior que o recomendado pela entidade NIH-US.

In [69]:
SELECT Origem
FROM NaMedioPorRefeicaoEmMgPorRegiao
    WHERE (3 * MediaSodio) > (SELECT RE.QuantidadeMg
                              FROM Recomendacao RE
                              JOIN Componente CO
                                  ON RE.IdComponente = CO.Id AND RE.Tipo = CO.Tipo
                              WHERE RE.Orgao = 'NIH_US' AND RE.FaixaEtaria = '19-30'
                                  AND CO.Nome = 'Sodium'
                              );


Em geral, no mundo os consumos de sódio estão muito altos. Estima-se que mais de 2 milhões de pessoas morrem, por ano, por causa de altos consumos desse componente. Essa tendência também é indicada nos dados que, apesar de não representarem necessariamente os consumos de cada população, são capazes de mostrar tendências.